# FIT5196 Task 2 in Assessment 1
#### Student Name: Swaraj Purohit
#### Student ID: 29286174



Environment: Python 3 and Jupyter notebook

Libraries used: 

* re (for regular expression, included in Anaconda Python 3) 
* nltk (for text analysis, included in Anaconda Python 3) 
* sklearn.feature_extraction.text (for count vectorizer, included in Anaconda Python 3)

note: The cells need to be run in order.

# Task 2 

### Getting required files

The text file has been modified a bit manually. Everything except the numbers needed for my ID is deleted from the file.

In [1]:
import re

In [2]:
# Getting res_nos, the list containing resume numbers needed. Duplicate resumes are removed.
f1 = open('resume_dataset.txt', 'r')
res_nos_txt = f1.read()
res_nos_txt
R = r'(\d.{0,3}?)\s'
res_nos = re.findall(R, res_nos_txt)
f1.close()
res_nos = list(set(res_nos))
len(res_nos)

220

In [3]:
# Getting text 0f required resumes in a dictionary format with resume number as key.
tokens_dict = {}
for num in res_nos:
    file = open('resume_('+ num +').txt', 'r', encoding = 'utf-8')
    res = file.read()
    tokens_dict[num] = res
    file.close()

### Generating dictionary of all tokens.

The dictionary will have all tokens for each resume with resume index as key. Stopwords and words tokens with length less than 3 are removed.

In [4]:
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import sent_tokenize

In [5]:
# Getting list of stopwords
file = open('stopwords_en.txt', 'r')
stopwords = file.read()
file.close()
stopword_list = stopwords.split('\n')

In [6]:
#Function to lower first word of each sentence in a resume.
def lowerer(in_str):
    sentences = sent_tokenize(in_str)
    out_str = ''
    for i in range(len(sentences)):
        sentences[i] = sentences[i].split()
        sentences[i][0] = sentences[i][0].lower()
        sentences[i] = ' '.join(sentences[i])
        out_str = ' '.join(sentences)
    return out_str

In [7]:
# Getting dictionary with list of all tokens as values for a resume number as key. First word of each sentence is lower cased.
#Stopwords are removed and tokens with length < 3 removed.

for k,v in tokens_dict.items():
    v = lowerer(v)
    tokenizer = RegexpTokenizer(r"\w+(?:[-.]\w+)?")
    unigram_tokens = tokenizer.tokenize(v)
    unigram_tokens = [tok for tok in unigram_tokens if tok not in stopword_list and len(tok) > 2]
    tokens_dict[k] = unigram_tokens

### Creating Vocab and Dictionary of all tokens.

Bigrams are incorporated, context dependant stopwords and rare tokens are removed. Then the Porter Stemmer is used. Changes are incorporated in both the vocab and the dictionary of tokens as values and resume number as keys.

In [8]:
import nltk

In [9]:
#Finding top200 bigrams.The frequency filter has been reduced to 14 because a higher filter gave less than 200 bigrams.

from itertools import chain
all_words = list(chain.from_iterable(tokens_dict.values()))

bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words)
bigram_finder.apply_freq_filter(14)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-100 bigrams
top_200_bigrams
len(top_200_bigrams)

200

In [10]:
#Incorporating bigrams in new dictionary of tokens called colloc_resumes. Initial vocab called colloc_voc is created.

from nltk.tokenize import MWETokenizer
mwetokenizer = MWETokenizer(top_200_bigrams)
colloc_resumes =  dict((index, mwetokenizer.tokenize(resume)) for index,resume in tokens_dict.items())
all_words_colloc = list(chain.from_iterable(colloc_resumes.values()))
colloc_voc = list(set(all_words_colloc))
print(len(colloc_voc))

16445


In [11]:
#Function that tells how many documents a token has appeared in.

def doc_counter(token, doc_dict):
    count = 0
    for each in doc_dict.values():
        if token in each:
            count += 1
    return count  

In [12]:
#Filtering out context dependant stopwords and rare tokens from inital vocab.

voc = [word for word in colloc_voc if 5 <= doc_counter(word, colloc_resumes) <=215]
len(voc)

3117

In [13]:
#Removing the context dependant stopwords and raretokens from the token dictionary.

for k,v in colloc_resumes.items():
    colloc_resumes[k] = [tok for tok in v if tok in voc]

In [14]:
#Stemming the tokens dict
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
for k,v in colloc_resumes.items():
    for i in range(len(colloc_resumes[k])):
        colloc_resumes[k][i] = stemmer.stem(colloc_resumes[k][i])

In [15]:
#Getting final vocab and creating a dictionary which maps the words to numbers
words = list(chain.from_iterable(colloc_resumes.values()))
vocab = list(set(words))
vocab.sort()
vocab_dict = dict((word,str(vocab.index(word))) for word in vocab)
vocab_dict

{'000': '0',
 '1.5': '1',
 '100': '2',
 '120': '3',
 '150': '4',
 '1984': '5',
 '1987': '6',
 '1989': '7',
 '1990': '8',
 '1991': '9',
 '1993': '10',
 '1995': '11',
 '1996': '12',
 '1997': '13',
 '1998': '14',
 '1999': '15',
 '1st': '16',
 '200': '17',
 '2000': '18',
 '2001': '19',
 '2002': '20',
 '2003': '21',
 '2004': '22',
 '2005': '23',
 '2006': '24',
 '2007': '25',
 '2008': '26',
 '2008_2009': '27',
 '2009': '28',
 '2009_2010': '29',
 '2010': '30',
 '2010_2011': '31',
 '2011': '32',
 '2012': '33',
 '2012_jun': '34',
 '2013': '35',
 '2014': '36',
 '2014_present': '37',
 '2015': '38',
 '2015_present': '39',
 '2016': '40',
 '2016_present': '41',
 '2017': '42',
 '2nd': '43',
 '3.8': '44',
 '300': '45',
 '3rd': '46',
 '4.0': '47',
 '4.00': '48',
 '400': '49',
 '500': '50',
 '6.5': '51',
 '800': '52',
 '852': '53',
 'abil': '54',
 'abl': '55',
 'absenc': '56',
 'academ': '57',
 'acca': '58',
 'accept': '59',
 'access': '60',
 'accord': '61',
 'account': '62',
 'account_open': '63',
 'ac

### Mapping words to their respective numbers.

In [16]:
# Function to map words to numbers using the vocab dict created above.
def word_to_num(lst):
    for i in range(len(lst)):
        for k,v in vocab_dict.items():
            if lst[i] == k:
                lst[i] = v
    return lst            

In [17]:
# Using the function to map the dictionary of all tokens and the vocab.
for k, v in colloc_resumes.items():
    colloc_resumes[k] = word_to_num(colloc_resumes[k])

vocab = word_to_num(vocab)

In [275]:
len(vocab)

1705

### Creating the required files.

In [18]:
# Creating list of strings of all resumes after they have been processed and mapped to numbers. 
# Creating list which holds the respective resume numbers.
clean_res_list = []
res_num = []
for k,v in colloc_resumes.items():   
    txt = ' '.join(v)
    clean_res_list.append(txt)
    res_num.append(k)   

In [19]:
# Creating the CountVectorizer. The columns are numbers which have been obtained by mapping the words.
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = 'word', input ='content', vocabulary = vocab)
data_features = vectorizer.fit_transform(clean_res_list)
print (data_features.shape)

(220, 1705)


In [31]:
# Writing the count vector file
file = open('29286174_countVec.txt', 'w')
ar = data_features.toarray()
for i in range(ar.shape[0]):
    file.write('resume_({0}), '.format(res_num[i]))
    for word, count in zip(vocab, ar[i]):
        
        if count > 0:
            file.write(word+':'+str(count)+', ')

    file.write('\n')
file.close()            

In [32]:
# Writing the vocab file
file = open('29286174_vocab.txt', 'w')
for k,v in vocab_dict.items():
    file.write(k+' : '+v+'\n')
file.close()    